In [58]:
import theano
# theano.config.optimizer = 'None' 
# theano.config.exception_verbosity='high'
print theano.config.optimizer
print theano.config.exception_verbosity
import theano.tensor as T
import numpy as np

fast_run
low


In [59]:
# DIMENSIONS = 5

In [60]:
# ETA = 0.5

In [61]:
# def setEigenSoFar(self,idx, w, v):
#     self.i = idx
#     for j in range(self.i):
#         self.eigenValues[j] = w[j]
#         self.eigenVectors[:,j] = v[:,j]


In [62]:
X_data = np.array(
[[ 0.41939831,  0.73336226,  0.81437075,  0.72610056,  0.76305652,],
 [ 0.66616774,  0.60886431,  0.92339832,  0.1030873 ,  0.31538147,],
 [ 0.04194576,  0.29987422,  0.7813136,   0.55106729,  0.58304852,],
 [ 0.05681328,  0.09614379,  0.83366364,  0.37384903,  0.06315704,],
 [ 0.97372508,  0.61315203,  0.2581135,   0.44524547,  0.21198589,],]
, dtype=np.float32
)
print X_data

[[ 0.41939831  0.73336226  0.81437075  0.72610056  0.76305652]
 [ 0.66616774  0.60886431  0.92339832  0.1030873   0.31538147]
 [ 0.04194576  0.29987422  0.7813136   0.55106729  0.58304852]
 [ 0.05681328  0.09614379  0.83366364  0.37384903  0.06315704]
 [ 0.97372508  0.61315203  0.2581135   0.44524547  0.21198589]]


In [65]:
class GradientDescent():
    def __init__(self):
        return

    def __init__(self, dim=5, eta=0.5, numEig=5):
        self.DIMENSIONS = dim
        self.ETA = eta
        self.NUM_EIG = numEig
        self.i = 0
        self.eigenValues = np.zeros(self.NUM_EIG, dtype=np.float32)
        self.eigenVectors = np.zeros((self.DIMENSIONS,self.NUM_EIG), dtype=np.float32)
        #         self.i = 0
        #         self.eigenValues = theano.shared( np.zeros(self.DIMENSIONS, dtype=np.float32) )
        #         self.eigenVectors = theano.shared( np.zeros((self.DIMENSIONS,self.DIMENSIONS), dtype=np.float32) )

    def compileTrainer(self):
        self.v = theano.shared( np.empty(self.DIMENSIONS, dtype=np.float32), name="v")
        self.X = T.fmatrix(name='X')
        #     self.X=X

        X_dot_v = T.dot(self.X,self.v)
        print 'X_dot_v', # theano.pp(X_dot_v)

        #         previousEigenContribution = \
        #         np.sum(
        #             [self.eigenValues[j]
        #              * T.dot(self.eigenVectors[j], self.v)
        #              * T.dot(self.eigenVectors[j], self.v)
        #                          for j in xrange(self.DIMENSIONS)])
        #         eigv_dot_v = T.dot(self.eigenVectors,self.v)
        #         previousEigenContribution = T.dot(self.eigenValues,(eigv_dot_v*eigv_dot_v))
        previousEigenContribution = np.sum( self.eigenValues[j]
                                           *T.dot(self.eigenVectors[:,j], self.v)
                                           *T.dot(self.eigenVectors[:,j], self.v)
                                               for j in xrange(self.NUM_EIG) )
        print 'previousEigenContribution', # theano.pp(previousEigenContribution)

        # np.sum(gd.eigenVectors[:,j]* gd.v for j in range(gd.i)).eval()

        self.objectiveFunction = T.dot(X_dot_v.T, X_dot_v) - previousEigenContribution
        print 'self.objectiveFunction', # theano.pp(self.objectiveFunction)

        grad_v = T.grad(self.objectiveFunction, self.v)
        print 'grad_v', # theano.pp(grad_v)

        y = self.v + self.ETA * grad_v
        updateFunction = y / y.norm(2)
        print 'y', # theano.pp(y)
        print 'updateFunction', # theano.pp(updateFunction)

        self.train = theano.function(
            inputs=[self.X],
            outputs=[updateFunction, grad_v, self.objectiveFunction, previousEigenContribution],
            updates=((self.v, updateFunction),)
        )
        print 'self.train trained'#, theano.pp(self.train)
        
    def getNextEigenValueAndVector(self, X_data, epsilon=1e-100, max_iteraions = 20, validateEpsilon=1):
        initVector = np.random.rand(self.DIMENSIONS).astype('float32')
        self.v.set_value(initVector)
        print 'Getting eigen value and vector #', self.i #, ' for: X = \n', X_data
        # print 'Starting with a random vector\n', v.get_value()
        last_v_star = self.v.get_value()
        checkEpsilon = validateEpsilon
        for i in range(max_iteraions):
            (update, grad, objective, contrib) = self.train(X_data)
            # print update
            #             print eigv_dot_v
            # print contrib
            # print update, grad, objective, contrib
            # self.eigenVectors[:,self.i] = update
            rms = sum(update - last_v_star)**2
            # print 'rms difference: ', rms
            last_v_star = update
            if(rms < epsilon):
                checkEpsilon-=1
                if(checkEpsilon<=0):
                    break
            else:
                checkEpsilon=validateEpsilon
        print 'Number of iterations done: ', i
        self.eigenValues[self.i] = self.objectiveFunction.eval({self.X:X_data})
        #         currEigenValues = self.eigenValues.get_value()
        #         currEigenValues[self.i] = self.objectiveFunction.eval({self.X:X_data})
        #         self.eigenValues.set_value( currEigenValues )

        self.eigenVectors[:,self.i] = update
        #         currEigenVectors = self.eigenVectors.get_value()
        #         currEigenVectors[:,self.i] = last_v_star
        #         self.eigenVectors.set_value( currEigenVectors )

        #         print "Eigen Value:", currEigenValues[self.i]
        #         print "Eigen Vector:\n", currEigenVectors[:,self.i]
        #         self.i+=1
        #         return currEigenValues[self.i-1], currEigenVectors[:,self.i-1]

        print "Eigen Value:", self.eigenValues[self.i]
        print "Eigen Vector:\n", self.eigenVectors[:,self.i]
        self.i+=1
        return self.eigenValues[self.i-1], self.eigenVectors[:,self.i-1]

    def getAllEigenValuesAndVectors(self, X_data, epsilon=1e-100, max_iteraions = 20, validateEpsilon=1):
        for e in range(self.NUM_EIG):
            self.compileTrainer()
            self.getNextEigenValueAndVector(X_data, epsilon=epsilon, max_iteraions=max_iteraions, validateEpsilon=validateEpsilon)
        return self.eigenValues, self.eigenVectors


In [67]:
gd1 = GradientDescent( eta=0.5, numEig=3)

In [68]:
gd1.getAllEigenValuesAndVectors(X_data, epsilon=1e-30, max_iteraions=50, validateEpsilon=5)

X_dot_v previousEigenContribution (((TensorConstant{0} + ((TensorConstant{0.0} * (TensorConstant{(5,) of 0.0} \dot v)) * (TensorConstant{(5,) of 0.0} \dot v))) + ((TensorConstant{0.0} * (TensorConstant{(5,) of 0.0} \dot v)) * (TensorConstant{(5,) of 0.0} \dot v))) + ((TensorConstant{0.0} * (TensorConstant{(5,) of 0.0} \dot v)) * (TensorConstant{(5,) of 0.0} \dot v)))
self.objectiveFunction grad_v y updateFunction self.train trained
Getting eigen value and vector # 0  for: X = 
[[ 0.41939831  0.73336226  0.81437075  0.72610056  0.76305652]
 [ 0.66616774  0.60886431  0.92339832  0.1030873   0.31538147]
 [ 0.04194576  0.29987422  0.7813136   0.55106729  0.58304852]
 [ 0.05681328  0.09614379  0.83366364  0.37384903  0.06315704]
 [ 0.97372508  0.61315203  0.2581135   0.44524547  0.21198589]]
Eigen Value: 6.69674
Eigen Vector:
[ 0.38626757  0.43620911  0.61527091  0.38440156  0.36632782]
X_dot_v previousEigenContribution (((TensorConstant{0} + ((TensorConstant{6.69674348831} * (TensorConstan

(array([ 6.69674349,  0.91701651,  0.35789928], dtype=float32),
 array([[ 0.38626757,  0.77879918, -0.18951496],
        [ 0.43620911,  0.3049418 ,  0.12934482],
        [ 0.61527091, -0.48440343, -0.62004143],
        [ 0.38440156, -0.17088737,  0.53058237],
        [ 0.36632782, -0.19139816,  0.53045732]], dtype=float32))

In [69]:
# w_np,v_np = np.linalg.eigh( np.dot(X_data.T,X_data) )
print X_data
w_np,v_np = np.linalg.eigh(np.dot(X_data.T,X_data))
idx = np.argsort(-w_np)
w_np = w_np[idx]
v_np = v_np[:,idx]
# for i in range(DIMENSIONS):
#     print w_np[i]
#     print v_np[:,i]
print w_np.astype('float32')
print v_np.astype('float32')


[[ 0.41939831  0.73336226  0.81437075  0.72610056  0.76305652]
 [ 0.66616774  0.60886431  0.92339832  0.1030873   0.31538147]
 [ 0.04194576  0.29987422  0.7813136   0.55106729  0.58304852]
 [ 0.05681328  0.09614379  0.83366364  0.37384903  0.06315704]
 [ 0.97372508  0.61315203  0.2581135   0.44524547  0.21198589]]
[  6.69674349e+00   9.17016327e-01   3.57899129e-01   1.17424600e-01
   5.43444697e-03]
[[-0.3862676  -0.77879918 -0.18949656 -0.17551515 -0.42137238]
 [-0.43620914 -0.30494162  0.12930381  0.3579978   0.75620782]
 [-0.61527091  0.48440337 -0.62004066 -0.01953718 -0.04430548]
 [-0.38440156  0.1708875   0.53066385 -0.72879076  0.10145342]
 [-0.36632782  0.19139819  0.5303933   0.5563401  -0.48820004]]


----------------------------
## Test single step

In [52]:
gd1.compileTrainer()

X_dot_v previousEigenContribution (((TensorConstant{0} + ((TensorConstant{0.0} * (TensorConstant{(5,) of 0.0} \dot v)) * (TensorConstant{(5,) of 0.0} \dot v))) + ((TensorConstant{0.0} * (TensorConstant{(5,) of 0.0} \dot v)) * (TensorConstant{(5,) of 0.0} \dot v))) + ((TensorConstant{0.0} * (TensorConstant{(5,) of 0.0} \dot v)) * (TensorConstant{(5,) of 0.0} \dot v)))
self.objectiveFunction grad_v y updateFunction self.train trained


In [53]:
gd1.getNextEigenValueAndVector(X_data, max_iteraions=100, epsilon=1e-50, validateEpsilon=10)

Getting eigen value and vector # 0  for: X = 
[[ 0.41939831  0.73336226  0.81437075  0.72610056  0.76305652]
 [ 0.66616774  0.60886431  0.92339832  0.1030873   0.31538147]
 [ 0.04194576  0.29987422  0.7813136   0.55106729  0.58304852]
 [ 0.05681328  0.09614379  0.83366364  0.37384903  0.06315704]
 [ 0.97372508  0.61315203  0.2581135   0.44524547  0.21198589]]
[ 0.45982906  0.44589075  0.51229298  0.39309907  0.41565335]
0.0
rms difference:  0.930760143038
[ 0.40474829  0.44200686  0.59409225  0.38568959  0.3729701 ]
0.0
rms difference:  0.000743005538127
[ 0.39086437  0.4379791   0.61067843  0.38441151  0.36701357]
0.0
rms difference:  7.32751623298e-05
[ 0.38741863  0.43668219  0.61424249  0.38433728  0.3663421 ]
0.0
rms difference:  3.70295417529e-06
[ 0.38655666  0.43632978  0.615035    0.38437244  0.36630604]
0.0
rms difference:  1.78739330892e-07
[ 0.38634017  0.43623936  0.61521572  0.38439184  0.36631823]
0.0
rms difference:  8.9477545373e-09
[ 0.38628578  0.43621662  0.6152578 

(6.6967435,
 array([ 0.38626757,  0.43620911,  0.61527091,  0.38440156,  0.36632782], dtype=float32))

In [54]:
print gd1.i
print gd1.eigenValues
print gd1.eigenVectors
# print gd1.eigenValues.get_value()
# print gd1.eigenVectors.get_value()

1
[ 6.69674349  0.          0.        ]
[[ 0.38626757  0.          0.        ]
 [ 0.43620911  0.          0.        ]
 [ 0.61527091  0.          0.        ]
 [ 0.38440156  0.          0.        ]
 [ 0.36632782  0.          0.        ]]


In [55]:
gd1.compileTrainer()
gd1.getNextEigenValueAndVector(X_data, max_iteraions=100, epsilon=1e-50, validateEpsilon=10)

X_dot_v previousEigenContribution (((TensorConstant{0} + ((TensorConstant{6.69674348831} * (TensorConstant{[ 0.386267...36632782]} \dot v)) * (TensorConstant{[ 0.386267...36632782]} \dot v))) + ((TensorConstant{0.0} * (TensorConstant{(5,) of 0.0} \dot v)) * (TensorConstant{(5,) of 0.0} \dot v))) + ((TensorConstant{0.0} * (TensorConstant{(5,) of 0.0} \dot v)) * (TensorConstant{(5,) of 0.0} \dot v)))
self.objectiveFunction grad_v y updateFunction self.train trained
Getting eigen value and vector # 1  for: X = 
[[ 0.41939831  0.73336226  0.81437075  0.72610056  0.76305652]
 [ 0.66616774  0.60886431  0.92339832  0.1030873   0.31538147]
 [ 0.04194576  0.29987422  0.7813136   0.55106729  0.58304852]
 [ 0.05681328  0.09614379  0.83366364  0.37384903  0.06315704]
 [ 0.97372508  0.61315203  0.2581135   0.44524547  0.21198589]]
[ 0.08179311  0.46953464 -0.08456524  0.74217641  0.4635407 ]
4.90696430206
rms difference:  0.213686477769
[ 0.0471461   0.41959554 -0.21662676  0.73355669  0.48650211]


(0.91701639,
 array([ 0.77879912,  0.30494177, -0.4844034 , -0.17088738, -0.19139814], dtype=float32))

In [56]:
print gd1.i
print gd1.eigenValues
print gd1.eigenVectors
# print gd1.eigenValues.get_value()
# print gd1.eigenVectors.get_value()

2
[ 6.69674349  0.91701639  0.        ]
[[ 0.38626757  0.77879912  0.        ]
 [ 0.43620911  0.30494177  0.        ]
 [ 0.61527091 -0.4844034   0.        ]
 [ 0.38440156 -0.17088738  0.        ]
 [ 0.36632782 -0.19139814  0.        ]]


In [ ]:
# gd=gd1


In [ ]:
# temp = gd.eigenVectors.eval()
# temp[:,1] = 0
# temp
# gd.eigenVectors.set_value(temp)


In [ ]:
# print gd.v.eval()
# print gd.eigenVectors.eval()
# print np.sum(gd.eigenVectors[:,j]* gd.v for j in range(gd.i)).eval()
# print gd.i
# print
# print (gd.eigenVectors*gd.v).eval()
# print ((gd.eigenVectors*gd.v)).eval()

In [ ]:
# print (T.sum(gd.eigenVectors*gd.v)).eval()
# print (T.sum(gd.eigenVectors*gd.v, axis=0)).eval()
# print (T.sum(gd.eigenVectors*gd.v, axis=1)).eval()

In [ ]:
# print T.dot(gd.eigenVectors[1], gd.v).eval()
# print gd.v.eval()
# print gd.eigenVectors[1].eval()
# print (T.dot(gd.eigenVectors[1], gd.v)**2).eval()
# print (gd.eigenValues[1]*(T.dot(gd.eigenVectors[1], gd.v)**2)).eval()
# print ((T.dot(gd.eigenVectors[1], gd.v))).eval()
# print (T.dot(gd.eigenVectors,gd.v)).eval()
# print ((T.dot(gd.eigenVectors[1], gd.v)**2)).eval()
# print (T.dot(gd.eigenVectors,gd.v)**2).eval()
# print (gd.eigenValues[1]*(T.dot(gd.eigenVectors[1], gd.v)**2)).eval()
# print (gd.eigenValues*(T.dot(gd.eigenVectors,gd.v)**2)).eval()
# print T.dot(gd.eigenValues,(T.dot(gd.eigenVectors,gd.v)**2)).eval()
# print (gd.eigenValues(T.dot(gd.eigenVectors[1], gd.v)**2)).eval()
# # T.sum?


---------------------------------------------------------------------------

In [44]:
theano.__version__

'0.9.0dev2.dev-26445969216722c8bc23548b6a39f26033caba39'